In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import SubgraphResults from './SubgraphResults';
import StockChart from './StockChart';

const useStyles = makeStyles({
  // ... (keep existing styles)
  chartContainer: {
    marginBottom: '20px',
    height: '400px',
  },
});

const EmailGenerator = () => {
  const styles = useStyles();
  const [chatMessages, setChatMessages] = useState([]);
  const [subgraphResults, setSubgraphResults] = useState(null);
  const [chatInput, setChatInput] = useState("");
  const [isChatActive, setIsChatActive] = useState(false);
  const [showChart, setShowChart] = useState(false);
  const [currentStep, setCurrentStep] = useState("");
  const [stockData, setStockData] = useState([]);
  const finalResponseRef = useRef(null);

  // ... (keep other existing state variables and functions)

  const handleUpdate = useCallback((data) => {
    if (data.type === 'step') {
      setCurrentStep(data.content);
    } else if (data.type === 'subgraph_results') {
      setSubgraphResults(data.content);
    } else if (data.type === 'final') {
      finalResponseRef.current = data.content;
      setChatMessages(prevMessages => {
        // Check if the last message is already the final response
        const lastMessage = prevMessages[prevMessages.length - 1];
        if (lastMessage && lastMessage.role === "assistant" && lastMessage.content === wrapInHtml(data.content)) {
          return prevMessages; // No need to add the final response again
        }
        const newMessages = [
          ...prevMessages,
          { role: "assistant", content: wrapInHtml(data.content) }
        ];
        // Show the chart after the first assistant message is added
        if (!showChart) {
          setShowChart(true);
        }
        return newMessages;
      });
      console.log("Received final response:", data.content);
    }
  }, [showChart, wrapInHtml]);

  // ... (keep other existing functions)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <>
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                  {index === chatMessages.length - 1 && message.role === "assistant" && (
                    <Button 
                      className={styles.exportButton}
                      onClick={handleExportToEmail}
                      icon={<FluentIcons.Mail24Regular />}
                    >
                      Export to Email
                    </Button>
                  )}
                </>
              )}
            </div>
          ))}
          {subgraphResults && (
            <div className={styles.subgraphMessage}>
              <SubgraphResults results={subgraphResults} />
            </div>
          )}
          {showChart && (
            <div className={styles.chartContainer}>
              <StockChart data={stockData} />
            </div>
          )}
        </div>
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (keep the rest of the component)
};

export default EmailGenerator;